In [3]:
import pandas as pd
gids = ['1164376473','1840297911','897421023','1167027454']
sheet_url = "https://docs.google.com/spreadsheets/d/1Ycg7zTxds9DZnDvTrFcyNNKuTUxg6Yy6WF0a8Wc02WQ/edit#gid="
url_1 = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')

transaction = pd.read_csv(url_1+gids[0])
clients = pd.read_csv(url_1+gids[1])
managers = pd.read_csv(url_1+gids[2])
leads = pd.read_csv(url_1+gids[3])

In [4]:
import numpy as np
social_source_utm = list(set(leads.d_utm_source.to_list()))
social_source_utm.remove('ycard#!/tproduct/225696739-1498486363994')
new_social_source = []
for i in social_source_utm:
  if type(i) == float:
    continue
  new_social_source.append(i)
social_source_utm = new_social_source
print(social_source_utm)

['insta', 'instagram', 'sms', 'viber', 'utm_source', 'ycard', 'vkontakte', 'google', 'ig', 'yandex', 'vk']


In [5]:
list_of_managers = managers.manager_id.to_list()
list_club_ids = managers.d_club.to_list()

In [6]:
list_of_clients = list(set(clients.client_id.to_list()))
clients.created_at = pd.to_datetime(clients.created_at,errors = 'coerce')

In [7]:
transaction.created_at = pd.to_datetime(transaction.created_at)
transaction = transaction.sort_values('created_at')

In [24]:
#I would consider the first index of the zeros as Trash(Мусорная заявка)
trash = leads.l_manager_id.value_counts().index[0]
manager_list = managers.manager_id.to_list()
club_list = managers.d_club.to_list()

######## Some Metrics and Results ########
total_trash_requests = 0
metric_results = []
for i in range(len(social_source_utm)):
  lists = []
  metric_results.append(lists)


leads['created_at'] = pd.to_datetime(leads['created_at'])
# Timeline 1 month 
sorted_month_leads = leads[(leads.created_at.dt.year == 2020) & (leads.created_at.dt.month == 10)]
sorted_month_leads.reset_index(inplace=True)
sorted_month_leads = sorted_month_leads.drop('index',axis=1)


n = 0
for i in range(len(metric_results)):
  vals = sorted_month_leads[(sorted_month_leads.d_utm_source == social_source_utm[i])]
  each_sets = list(set(vals.l_manager_id.to_list()))
  temp_list = []
  for j in each_sets:
    totals = {
      'Канал привлечения заявки' : social_source_utm[i],
      'Клуб' : "",
      'Менеджер' : "",
      'Количество заявок' : 0,
      'Количество мусорных заявок' : 0,
      'Количество новых заявок' : 0,
      'Количество покупателей' : 0,
      'Количество новых покупателей' : 0,
      'Доход от покупок новых покупателей' : 0
    }
    if j == trash:
      continue
    elif j not in list_of_managers:
      totals['Клуб'] = ""
      totals['Менеджер'] = ""
    else:
      index = list_of_managers.index(j)
      totals['Клуб'] = list_club_ids[index]
      totals['Менеджер'] = managers.d_manager[i]
    totals['Количество заявок'] = len(vals[(vals.l_client_id != trash)])
    trashes = vals[(vals.l_client_id == trash)]
    totals['Количество мусорных заявок'] = len(trashes.l_client_id.to_list())
    n_req = vals[(vals.l_client_id != trash) & any(item not in vals.l_client_id.to_list() for item in list_of_clients)]
    totals['Количество новых заявок'] = len(n_req)
    ix = vals[(vals.l_client_id != trash) & any(item in vals.l_client_id.to_list() for item in list_of_clients)].l_client_id.to_list()
    number_of_buyers = 0
    number_of_newbuyers = 0
    revenue_of_newbuyers = 0
    for k in ix:
      client_transaction = transaction[(transaction.l_client_id == k)]

      client_days = clients.created_at.dt.day*24*60*60
      client_hours = clients.created_at.dt.hour*60*60
      client_minutes = clients.created_at.dt.minute*60
      client_seconds = clients.created_at.dt.second
      client_total = client_days + client_hours + client_minutes + client_seconds
      days = client_transaction.created_at.dt.day*24*60*60
      created_days = vals[vals.l_client_id == k].created_at.dt.day*24*60*60
      hours = client_transaction.created_at.dt.hour*60*60
      created_hours = vals[vals.l_client_id == k].created_at.dt.hour*60*60
      minutes = client_transaction.created_at.dt.minute*60
      created_minutes = vals[vals.l_client_id == k].created_at.dt.minute*60
      seconds = client_transaction.created_at.dt.second
      created_seconds = vals[vals.l_client_id == k].created_at.dt.second
      created_total = created_days+created_hours+created_minutes+created_seconds
      total = days+hours+minutes+seconds
      division = abs(client_total-created_total).iloc[0].astype(int)
      if(len(total) == 0):
        break
      if created_total.iloc[0] < total.iloc[0] and division < 604800:
        number_of_newbuyers+=1
        revenue_of_newbuyers+=sum(client_transaction.m_real_amount.to_list())
    iy = vals[(vals.l_client_id != trash) & any(item in vals.l_client_id.to_list() for item in list_of_clients)].l_client_id.to_list()

    for k in iy:
      client_transaction = transaction[(transaction.l_client_id == k)]
      days = client_transaction.created_at.dt.day*24*60*60
      created_days = vals[vals.l_client_id == k].created_at.dt.day*24*60*60
      hours = client_transaction.created_at.dt.hour*60*60
      created_hours = vals[vals.l_client_id == k].created_at.dt.hour*60*60
      minutes = client_transaction.created_at.dt.minute*60
      created_minutes = vals[vals.l_client_id == k].created_at.dt.minute*60
      seconds = client_transaction.created_at.dt.second
      created_seconds = vals[vals.l_client_id == k].created_at.dt.second

      created_total = created_days+created_hours+created_minutes+created_seconds
      total = days+hours+minutes+seconds
      if(len(total) == 0):
        break
      if created_total.iloc[0] > total.iloc[0]:
        number_of_buyers+=1
    totals['Количество покупателей'] = number_of_buyers
    totals['Количество новых покупателей'] = number_of_newbuyers
    totals['Доход от покупок новых покупателей'] = revenue_of_newbuyers
    temp_list.append(totals)
    
  metric_results[i] = temp_list[:]
print(metric_results[0])

[{'Канал привлечения заявки': 'insta', 'Клуб': 'club #2', 'Менеджер': 'manager #8', 'Количество заявок': 16, 'Количество мусорных заявок': 1, 'Количество новых заявок': 16, 'Количество покупателей': 0, 'Количество новых покупателей': 1, 'Доход от покупок новых покупателей': 66402}, {'Канал привлечения заявки': 'insta', 'Клуб': 'club #4', 'Менеджер': 'manager #8', 'Количество заявок': 16, 'Количество мусорных заявок': 1, 'Количество новых заявок': 16, 'Количество покупателей': 0, 'Количество новых покупателей': 1, 'Доход от покупок новых покупателей': 66402}, {'Канал привлечения заявки': 'insta', 'Клуб': 'club #1', 'Менеджер': 'manager #8', 'Количество заявок': 16, 'Количество мусорных заявок': 1, 'Количество новых заявок': 16, 'Количество покупателей': 0, 'Количество новых покупателей': 1, 'Доход от покупок новых покупателей': 66402}, {'Канал привлечения заявки': 'insta', 'Клуб': 'club #4', 'Менеджер': 'manager #8', 'Количество заявок': 16, 'Количество мусорных заявок': 1, 'Количество 

In [9]:
pip install pygsheets

     |████████████████████████████████| 153kB 6.5MB/s 


In [25]:
import pygsheets
import pandas as pd

gc = pygsheets.authorize(service_file='client_secret.json')
sh = gc.open('Metrics')

for i in range(len(metric_results)):
  wsk = sh[i]
  df = pd.DataFrame(metric_results[i])
  wsk.set_dataframe(df,(1,1),extend=True)